In [17]:
import numpy as np
import glob
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from tensorflow import keras

In [18]:
df = pd.read_csv("./Data/lstm.csv")
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


In [19]:
df['paragraph'][0]

'dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait'

In [20]:
df['category'].unique()

array(['food', 'sports'], dtype=object)

In [21]:
# 데이터에 사용된 중복 없는 전체 단어 개수를 파악
results = set()
df['paragraph'].str.lower().str.split().apply(results.update)
vocab_size = len(results)
vocab_size

536

In [22]:
# 단어를 숫자로 인코딩
paragraphs = df['paragraph'].to_list()
paragraphs[0:5]

['dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait',
 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations',
 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table',
 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition',
 'the biscuits and gravy was too salty two people in my group had the gravy and all thoug

In [23]:
keras.utils.set_random_seed(1)

In [28]:
# one-hot encoding
encoded_paragraphs = [keras.preprocessing.text.one_hot(paragraph, vocab_size) for paragraph in paragraphs]
encoded_paragraphs[0]

[206,
 419,
 140,
 284,
 114,
 196,
 396,
 419,
 406,
 409,
 268,
 308,
 278,
 256,
 412,
 145,
 450,
 158,
 215,
 176,
 42,
 201,
 194,
 506,
 443,
 191,
 384,
 186,
 158,
 301,
 285,
 483,
 182,
 291,
 20,
 148,
 375,
 77,
 112,
 395,
 487,
 384,
 220,
 243,
 305,
 180,
 108,
 52,
 371,
 268,
 214,
 158,
 163]

In [25]:
# 데이터에서 가장 긴 문장의 단어 개수를 확인
max_length = 0

for row in df['paragraph']:
    if len(row.split(" ")) > max_length:
        max_length = len(row.split(" "))
print(max_length)

91


In [29]:
# 문장마다 단어 개수가 다르므로 제로패딩을 넣어줘서 문장의 길이를 동일하게 만든다.
padded_paragraphs_encoding = keras.preprocessing.sequence.pad_sequences(encoded_paragraphs,maxlen=max_length,padding='post')
padded_paragraphs_encoding

array([[206, 419, 140, ...,   0,   0,   0],
       [267, 412, 145, ...,   0,   0,   0],
       [339, 231, 105, ...,   0,   0,   0],
       ...,
       [519, 209, 483, ...,   0,   0,   0],
       [278, 268, 226, ...,   0,   0,   0],
       [ 80, 186, 268, ...,   0,   0,   0]], dtype=int32)

In [31]:
# 분류항목을 수치로 변경하기
categories = df['category'].to_list()

def category_encode(category):
    if category == "food":
        return [1,0]
    else:
        return [0,1]

In [32]:
encoded_category = [category_encode(category) for category in categories]

In [33]:
print(encoded_category[0])
print(encoded_category[19])

[1, 0]
[0, 1]


In [35]:
# Feature 확인
print(encoded_paragraphs[0])
print(encoded_paragraphs[19])

[206, 419, 140, 284, 114, 196, 396, 419, 406, 409, 268, 308, 278, 256, 412, 145, 450, 158, 215, 176, 42, 201, 194, 506, 443, 191, 384, 186, 158, 301, 285, 483, 182, 291, 20, 148, 375, 77, 112, 395, 487, 384, 220, 243, 305, 180, 108, 52, 371, 268, 214, 158, 163]
[80, 186, 268, 77, 487, 142, 264, 77, 10, 360, 378, 458, 483, 497, 284, 268, 503, 448, 396, 77, 105, 8, 190, 481, 474, 158, 268, 510, 481, 242, 19, 396, 255, 483, 56, 77, 507, 42, 442, 398, 30, 20, 137, 512, 191, 139, 257, 518, 158, 381, 268, 256, 521, 278, 256, 83, 264, 525, 366, 67, 210, 173, 474, 188, 436, 463, 215, 77, 483, 20, 252, 248, 24]


### 주제를 분류하는 모델 구현하기

In [41]:
model = keras.Sequential()
# 문맥 생성단계
model.add(keras.layers.Embedding(vocab_size, 5, input_length=max_length))
model.add(keras.layers.LSTM(64))
# 분류 단계
model.add(keras.layers.Dense(32,activation='relu'))
model.add(keras.layers.Dense(2,activation='softmax'))
# [0,1] or [1,0]이므로 softmax

In [42]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics='accuracy')

In [43]:
train_X = np.array(padded_paragraphs_encoding)
train_Y = np.array(encoded_category)

In [44]:
model.fit(train_X,train_Y, batch_size=10,epochs=50)

Epoch 1/50
2/2 [==============================] - 1s 9ms/step - loss: 0.6943 - accuracy: 0.5000
Epoch 2/50
2/2 [==============================] - 0s 10ms/step - loss: 0.6929 - accuracy: 0.5000
Epoch 3/50
2/2 [==============================] - 0s 10ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 4/50
2/2 [==============================] - 0s 9ms/step - loss: 0.6928 - accuracy: 0.5000
Epoch 5/50
2/2 [==============================] - 0s 10ms/step - loss: 0.6927 - accuracy: 0.5500
Epoch 6/50
2/2 [==============================] - 0s 11ms/step - loss: 0.6924 - accuracy: 0.5500
Epoch 7/50
2/2 [==============================] - 0s 14ms/step - loss: 0.6923 - accuracy: 0.5500
Epoch 8/50
2/2 [==============================] - 0s 13ms/step - loss: 0.6919 - accuracy: 0.5500
Epoch 9/50
2/2 [==============================] - 0s 10ms/step - loss: 0.6916 - accuracy: 0.5500
Epoch 10/50
2/2 [==============================] - 0s 9ms/step - loss: 0.6915 - accuracy: 0.5500
Epoch 11/50
2/2 [===============

In [45]:
model.evaluate(train_X,train_Y)

1/1 [==============================] - 0s 221ms/step - loss: 0.4179 - accuracy: 0.8000


[0.4179399013519287, 0.800000011920929]